In [1]:
from DataPlug import DataPlug as dp
from SqueezeNet import SqueezeNet as sq

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jackk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
squeeze = sq("SqueezeNet")
dataPlug = dp("DataPlug")


In [3]:
data = dataPlug.get_reddit_dataframe()


display(dataPlug.df)

,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,1.611862e+09,NaN,2021-01-28
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,1.611862e+09,NaN,2021-01-28
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,1.611862e+09,NaN,2021-01-28
...,...,...,...,...,...,...,...,...
53182,What I Learned Investigating SAVA FUD Spreaders,238,owd2pn,https://www.reddit.com/r/wallstreetbets/commen...,87,1.627906e+09,***TLDR: Three bitter scientists partnered up ...,2021-08-02
53183,"Daily Popular Tickers Thread for August 02, 20...",228,owd1a5,https://www.reddit.com/r/wallstreetbets/commen...,1070,1.627906e+09,\nYour daily hype thread. Please keep the shit...,2021-08-02
53184,Hitler reacts to the market being irrational,7398,owc5dr,https://v.redd.it/46jxu074exe71,372,1.627902e+09,NaN,2021-08-02
53185,"Daily Discussion Thread for August 02, 2021",338,owbfjf,https://www.reddit.com/r/wallstreetbets/commen...,11688,1.627898e+09,Your daily trading discussion thread. Please k...,2021-08-02


In [4]:
dataPlug.get_price_dataframe()

,timestamp,Open,High,Low,Close,Adj Close,Volume
0,2020-09-01,1.8250,1.955000,1.6925,1.912500,1.912500,92634400
1,2020-09-02,1.9500,2.012500,1.7775,1.927500,1.927500,52044400
2,2020-09-03,1.9700,2.112500,1.8100,1.955000,1.955000,57378000
3,2020-09-04,1.9450,1.980000,1.7925,1.912500,1.912500,30648000
4,2020-09-08,1.8875,2.070000,1.8700,1.925000,1.925000,39266400
...,...,...,...,...,...,...,...
653,2023-04-06,22.0000,22.670000,21.7700,22.400000,22.400000,2506900
654,2023-04-10,21.8500,22.430000,21.8200,22.370001,22.370001,2176100
655,2023-04-11,22.5700,23.170000,22.4200,22.690001,22.690001,2540000
656,2023-04-12,22.9000,23.280001,22.2600,22.260000,22.260000,2141100


In [5]:

dataPlug.aggregate_reddit_posts_daily()



display(dataPlug.df)



,title,score,id,url,comms_num,created,body
timestamp,,,,,,,
2020-09-29,$CBAT $SUNW $SPI $OXBR Y'all are missing the p...,4.000000,j1fmmo,https://www.reddit.com/r/wallstreetbets/commen...,11.000000,1.601340e+09,$CBAT is going to be a great play further yet:...
2021-01-28,"It's not about the money, it's about sending a...",960.609023,l6ulcx||l6uibd||l6uhhn||l6ugk6||l6ufgy||l6uf7d...,https://v.redd.it/6j75regs72e61||https://v.red...,142.193818,1.611863e+09,NaN||NaN||The CEO of NASDAQ pushed to halt tra...
2021-01-29,Current data on GME short positions||ZOM||xspa...,408.457309,l6yxto||l6yxqf||l6yxjc||l6yx9l||l6yx9k||l6yx9c...,https://www.reddit.com/r/wallstreetbets/commen...,52.371288,1.611875e+09,Is there data on current short positions in GM...
2021-01-30,Fellow retards! Hold that shit. Don’t panic.||...,4618.594101,l7uyb5||l7uyae||l7uya6||l7uy9j||l7uy8r||l7uy8e...,https://www.reddit.com/r/wallstreetbets/commen...,515.766152,1.611962e+09,We got this! Buy more if you can||What's the t...
2021-01-31,WE ALL DID IT! CITRON IS THROWING IN THE TOWEL...,5230.513312,l8nv7s||l8o31w||l8oblj||l8obkv||l8obkm||l8obk3...,https://i.redd.it/rpase2bamhe61.jpg||https://w...,306.713525,1.612049e+09,NaN||I can’t help but feel that whilst it was ...
...,...,...,...,...,...,...,...
2021-08-12,Here you go you sick fucks $WISH||The guide to...,647.386667,p38oq7||p388pe||p3841a||p37krv||p371r9||p3719y...,https://i.redd.it/bmf0yo4fqzg71.jpg||https://w...,481.200000,1.628802e+09,NaN||As you are all aware the Haitian presiden...
2021-08-13,Would manny monitors help you make more or los...,773.593750,p3tl78||p3tino||p3sv76||p3sv17||p3s821||p3rjhp...,https://v.redd.it/w4lkj0upr6h71||https://i.red...,695.546875,1.628887e+09,NaN||NaN||Your weekend discussion thread. Plea...
2021-08-14,"My First 3 Months with WSB||Nice monitors, I u...",1204.837838,p4e21n||p4df57||p4chcx||p4cfun||p4cdx5||p4bji7...,https://v.redd.it/9ldnh4figdh71||https://v.red...,153.783784,1.628968e+09,NaN||NaN||NaN||📷\n\nhttps://preview.redd.it/4o...


In [36]:
# Merge the dataframes
dataPlug.merge_dataframes()


display(dataPlug.mergedDF)

,timestamp,Open,High,Low,Close,Adj Close,Volume,title,score,comms_num,body
0,2020-09-29,2.500000,2.662500,2.482500,2.587500,2.587500,20950400,$CBAT $SUNW $SPI $OXBR Y'all are missing the p...,4.000000,11.000000,$CBAT is going to be a great play further yet:...
1,2021-01-28,66.250000,120.750000,28.062500,48.400002,48.400002,235263200,"It's not about the money, it's about sending a...",960.609023,142.193818,NaN||NaN||The CEO of NASDAQ pushed to halt tra...
2,2021-01-29,94.927498,103.495003,62.500000,81.250000,81.250000,202264400,Current data on GME short positions||ZOM||xspa...,408.457309,52.371288,Is there data on current short positions in GM...
3,2021-02-01,79.139999,80.500000,53.000000,56.250000,56.250000,149528800,RH closing positions and limiting buying due t...,4230.302036,531.211538,This is another sign that we're definitely in ...
4,2021-02-02,35.189999,39.500000,18.555000,22.500000,22.500000,312732400,Bloomberg reports short interest down to 39% f...,786.005326,101.698402,NaN||NaN||The news is saying Reddit (which wou...
...,...,...,...,...,...,...,...,...,...,...,...
118,2021-08-10,40.340000,41.724998,38.837502,39.762501,39.762501,6493200,Follow The Money - How To Catch Every Rotation...,538.592593,630.555556,# I. Introduction\n\nSorry for the wall of tex...
119,2021-08-11,39.607498,39.762501,38.654999,39.695000,39.695000,3781600,GROW: the same guy who discovered GME in June ...,613.945055,365.296703,**Facts:**\n\n* Josh Klein is a deep-value in...
120,2021-08-12,39.970001,41.070000,39.332500,40.587502,40.587502,5271200,Here you go you sick fucks $WISH||The guide to...,647.386667,481.200000,NaN||As you are all aware the Haitian presiden...
121,2021-08-13,40.117500,40.887501,39.352501,40.630001,40.630001,4059200,Would manny monitors help you make more or los...,773.593750,695.546875,NaN||NaN||Your weekend discussion thread. Plea...


In [31]:
positive = []
negative = []
neutral = []
compound = []

import time

begin = time.time() # timer for entire process

for i in range(0, len(dataPlug.mergedDF['title'])):
    start = time.time() # timer for each iteration

    if i == 2:
        positive.append(0)
        negative.append(0)
        neutral.append(0)
        compound.append(0)
        continue

    text = dataPlug.mergedDF['title'][i]
    clean_text = squeeze.clean_text(text)
    score = squeeze.sentiment_analysis(clean_text)
    # Add scores to array
    positive.append(score['pos'])
    negative.append(score['neg'])
    neutral.append(score['neu'])
    compound.append(score['compound'])



    end = time.time() # end timer for each iteration

    print(f"Completed {i} of {len(dataPlug.mergedDF['title'])} posts | {end - start:0.4f} seconds | {end - begin:0.4f} seconds total")

print(f"Sentiment analysis took {end - begin:0.4f} seconds total")



Completed 0 of 123 posts | 0.0110 seconds | 0.0120 seconds total
Completed 1 of 123 posts | 7.8177 seconds | 7.8297 seconds total
Completed 3 of 123 posts | 8.6651 seconds | 16.4958 seconds total
Completed 4 of 123 posts | 25.6527 seconds | 42.1485 seconds total
Completed 5 of 123 posts | 75.4704 seconds | 117.6188 seconds total
Completed 6 of 123 posts | 20.4297 seconds | 138.0495 seconds total
Completed 7 of 123 posts | 43.2706 seconds | 181.3201 seconds total
Completed 8 of 123 posts | 0.5463 seconds | 181.8664 seconds total
Completed 9 of 123 posts | 0.1091 seconds | 181.9755 seconds total
Completed 10 of 123 posts | 0.2271 seconds | 182.2026 seconds total
Completed 11 of 123 posts | 0.1350 seconds | 182.3376 seconds total
Completed 12 of 123 posts | 0.1650 seconds | 182.5026 seconds total
Completed 13 of 123 posts | 0.0460 seconds | 182.5486 seconds total
Completed 14 of 123 posts | 0.1820 seconds | 182.7306 seconds total
Completed 15 of 123 posts | 0.4812 seconds | 183.2117 secon

In [37]:
# Using DataFrame.insert() to add the sentiment columns to the dataframe
dataPlug.mergedDF.insert(8, "Compound_Sentiment", compound, True)
dataPlug.mergedDF.insert(8, "Negative_Sentiment", negative, True)
dataPlug.mergedDF.insert(8, "Neutral_Sentiment", neutral, True)
dataPlug.mergedDF.insert(8, "Positive_Sentiment", positive, True)
print('Done!')

Done!


In [38]:
# Drop the column with the weird data issue
dataPlug.mergedDF = dataPlug.mergedDF.drop(labels=2, axis=0)

In [39]:
display(dataPlug.mergedDF)

,timestamp,Open,High,Low,Close,Adj Close,Volume,title,Positive_Sentiment,Neutral_Sentiment,Negative_Sentiment,Compound_Sentiment,score,comms_num,body
0,2020-09-29,2.500000,2.662500,2.482500,2.587500,2.587500,20950400,$CBAT $SUNW $SPI $OXBR Y'all are missing the p...,0.000,0.784,0.216,-0.2960,4.000000,11.000000,$CBAT is going to be a great play further yet:...
1,2021-01-28,66.250000,120.750000,28.062500,48.400002,48.400002,235263200,"It's not about the money, it's about sending a...",0.240,0.662,0.099,1.0000,960.609023,142.193818,NaN||NaN||The CEO of NASDAQ pushed to halt tra...
3,2021-02-01,79.139999,80.500000,53.000000,56.250000,56.250000,149528800,RH closing positions and limiting buying due t...,0.283,0.639,0.077,1.0000,4230.302036,531.211538,This is another sign that we're definitely in ...
4,2021-02-02,35.189999,39.500000,18.555000,22.500000,22.500000,312732400,Bloomberg reports short interest down to 39% f...,0.271,0.640,0.090,1.0000,786.005326,101.698402,NaN||NaN||The news is saying Reddit (which wou...
5,2021-02-03,28.002501,28.350000,21.312500,23.102501,23.102501,170794000,AXOS CASHAPP PARENT COMPANY SHUTS DOWN AMC AND...,0.260,0.648,0.092,1.0000,983.322361,128.741203,NaN||I know a big drop is scary. Robingay cut ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2021-08-10,40.340000,41.724998,38.837502,39.762501,39.762501,6493200,Follow The Money - How To Catch Every Rotation...,0.170,0.757,0.073,0.9949,538.592593,630.555556,# I. Introduction\n\nSorry for the wall of tex...
119,2021-08-11,39.607498,39.762501,38.654999,39.695000,39.695000,3781600,GROW: the same guy who discovered GME in June ...,0.164,0.781,0.055,0.9988,613.945055,365.296703,**Facts:**\n\n* Josh Klein is a deep-value in...
120,2021-08-12,39.970001,41.070000,39.332500,40.587502,40.587502,5271200,Here you go you sick fucks $WISH||The guide to...,0.210,0.723,0.066,0.9988,647.386667,481.200000,NaN||As you are all aware the Haitian presiden...
121,2021-08-13,40.117500,40.887501,39.352501,40.630001,40.630001,4059200,Would manny monitors help you make more or los...,0.169,0.732,0.099,0.9921,773.593750,695.546875,NaN||NaN||Your weekend discussion thread. Plea...


{'neg': 0.216, 'neu': 0.784, 'pos': 0.0, 'compound': -0.296}
